In [ ]:
import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as la
from scipy import linalg

sys.path.insert(0, '../../../methods')
from capon import *

In [ ]:
snr_list = [-20,-15,-10,-5,0,5,10,15,20,25,30]
T_list = [100,200,500,1000,2000,5000,10000]
rmse_list1 = []
rmse_list2 = []

for snr in snr_list:
    # load data
    data = np.load(f'../../../data/experiment_1/scenario_2/data_test_snr{snr}_t1000.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    data = [s['cm'] for s in data]

    truths = np.stack(truths).squeeze(2)
    data = np.stack(data)

    # apply capon beamformer
    preds = np.zeros((data.shape[0],4))

    for i in range(data.shape[0]):
        R = data[i,:,:]
        M = 8
        N = 4
        res = 1
        sensor_pos = np.zeros((M,3))
        sensor_pos[:,0] = np.array([1,2,3,4,5,6,11,16])*0.5

        preds[i,:], spectrum = capon(R,M,N,sensor_pos,res)
        preds[i] = np.sort(preds[i])
        truths[i] = np.sort(truths[i])
        
        if snr==10 and i==0:
             np.save(f'../../../results/experiment_1/scenario_2/capon_spectrum_{truths[0]}deg.npy', spectrum)

    rmse = np.sqrt(np.mean((truths-preds)**2))
    rmse_list1.append(rmse)
    
    print(f"snr {snr}dB, T {1000}, test-rmse {rmse:.4f}")
    
    np.save(f'../../../results/experiment_1/scenario_2/capon_preds_snr{snr}_t1000.npy', preds)
    np.save(f'../../../results/experiment_1/scenario_2/capon_truths_snr{snr}_t1000.npy', truths)
    
for T in T_list:
    # load data
    data = np.load(f'../../../data/experiment_1/scenario_2/data_test_snr-10_t{T}.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    data = [s['cm'] for s in data]

    truths = np.stack(truths).squeeze(2)
    data = np.stack(data)

    # apply capon beamformer
    preds = np.zeros((data.shape[0],4))

    for i in range(data.shape[0]):
        R = data[i,:,:]
        M = 8
        N = 4
        res = 1
        sensor_pos = np.zeros((M,3))
        sensor_pos[:,0] = np.array([1,2,3,4,5,6,11,16])*0.5

        preds[i,:], spectrum = capon(R,M,N,sensor_pos,res)
        preds[i] = np.sort(preds[i])
        truths[i] = np.sort(truths[i])

    rmse = np.sqrt(np.mean((truths-preds)**2))
    rmse_list2.append(rmse)
    
    print(f"snr -10dB, T {T}, test-rmse {rmse:.4f}")
    
    np.save(f'../../../results/experiment_1/scenario_2/capon_preds_snr-10_t{T}.npy', preds)
    np.save(f'../../../results/experiment_1/scenario_2/capon_truths_snr-10_t{T}.npy', truths)
    
np.save('../../../results/experiment_1/scenario_2/capon_rmse1.npy', rmse_list1)
np.save('../../../results/experiment_1/scenario_2/capon_rmse2.npy', rmse_list2)

In [ ]:
# plot rmse values
plt.figure()
plt.plot(snr_list, rmse_list1, '-o')
plt.title("rmse values for different snr levels")
plt.xlabel("snr (dB)")
plt.ylabel("rmse (deg)")
plt.legend(['capon'])
plt.yscale("log")
plt.grid()

plt.figure()
plt.plot(T_list, rmse_list2, '-o')
plt.title("rmse values for different snapshot numbers")
plt.xlabel("T")
plt.ylabel("rmse (deg)")
plt.legend(['capon'])
plt.yscale("log")
plt.grid()